In [1]:
from google.colab import drive, files
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
!pip install schedule
!cp /content/drive/MyDrive/AQI_Testing.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import json
import pandas as pd
import warnings
import time
from datetime import date, datetime
from dateutil.tz import gettz
import schedule
from pathlib import Path
from multiprocessing import Process
from AQI_Testing import test_cells
import os
import glob

ModuleNotFoundError: No module named 'AQI_Testing'

In [15]:
def getTime():
    now = datetime.now(tz=gettz('Asia/Kolkata'))
    current_time = now.strftime("%d-%b-%Y-%H-%M")
    return current_time
  
def getDate():
  return datetime.now(tz=gettz('Asia/Kolkata')).strftime("%Y-%m-%d")

In [17]:
AQIData_location = "/content/drive/MyDrive/AQIData/"
FileData_location = "/content/drive/MyDrive/Filtered_Data/"
Path(FileData_location).mkdir(parents=True,exist_ok=True)

In [18]:
def filter_data(
    folder_date, device_name_list = [],
    data_needed = ['PM25','PM10','Temp(cel)','Hum'],
    columns =['devicelat','devicelon']):
  #print('Start Filter')
  data_cols = {col:[] for col in data_needed}
  csv_files = glob.glob(os.path.join(AQIData_location+folder_date, "*.csv"))
  #print('File Opened')
  data1 = pd.DataFrame()
  for i,csv in enumerate(csv_files):
    df2 = pd.read_csv(csv)
    if device_name_list:
      df2 = df2.loc[(df2['devicename'].isin(device_name_list)) & (df2['isOnline'] == 1)]
    else:
      df2 = df2.loc[(df2['isOnline'] == 1)]
    if df2.empty:
      continue
    data1 = df2 if i==0 else data1.append(df2)
  #print('Files Appended')
  data1.reset_index(inplace=True)
  for j in range(len(data1)):
    realtime = data1['realtime'][j]
    res = list(eval(realtime))
    for entry in res:
      if entry['sensorname'] in data_needed:
        data_cols[entry['sensorname']].append(entry['sensorvalue'])
  #print('Data Extracted')

  print(f'Data Size: {len(data1)}')
  if len(data1)==0:
    print('No data to save, Exiting Filter Data')
    return
  data2 = pd.DataFrame()
  data2 = data1.loc[:,columns]
  for col in data_cols.keys():
    data2[col] = data_cols[col]
  data2 = data2.dropna()
  print(f'Columns saved: {data2.head(0)}')
  save_file = FileData_location + folder_date + '.csv'
  print(f"Saving Filtered Data: {save_file}")
  data2.to_csv(save_file, index=False)
# filter_data(getDate())

class newProcess(Process):
  def __init__(self):
    Process.__init__(self)
  def run(self, run_date, device_name_list = ["Outdoor 4",'Outdoor 8'], data_needed = ['PM25','PM10','Temp(cel)','Hum']):
    filter_data(run_date, device_name_list, data_needed)

# saveProcess = newProcess()

In [ ]:
# def get_token():
#     url = "https://secure.aqi.in/api/v1/login"
#     pswd="1234567" 
#     query = {'email':"iitr@aqi.in", 'password':pswd}
#     res = requests.post(url, headers={"Content-Type": "application/x-www-form-urlencoded"}, params=query)
#     return res.json()['data']["token"]

# url = "https://secure.aqi.in/api/v1/UserDeviceList"
# authHeader = "Authorization"
# res = requests.get(url,headers={authHeader: get_token()})

# print(json.dumps(res.json(),indent=2))

In [20]:
tk={'today': None,'token': None}
fileCount = 0
testCount = 5
isTesting = True
devices = []

def get_token(today):
    print("generating token on ", today)
    url = "https://secure.aqi.in/api/v1/login"
    pswd= "1234567" #pd.read_csv('D:\sem01_02\APIdata\pswd.csv', header=None).iloc[1,1]
    query = {'email':"iitr@aqi.in", 'password':pswd}
    res = requests.post(url, headers={"Content-Type": "application/x-www-form-urlencoded"}, params=query)
    return res.json()['data']["token"]

def get_realTimeData():
    global fileCount   
    url = "https://secure.aqi.in/api/v1/UserDeviceList"
    now = getDate()
    if (not tk['today'] == now and tk['today']) or (fileCount ==testCount and isTesting):
      fileCount = 0
      startTime = datetime.now()
      print('Filtering Data')
      filter_data(tk['today'], device_name_list = devices, columns = ['devicelat','devicelon', 'devicename'])
      print(f'Time_Taken: {datetime.now()-startTime}')
    if (tk['token'] == None) or not (tk['today'] == now):
        tk['token'] = get_token(now)
        tk['today'] = now
    res = requests.get(url,headers={"Authorization": tk['token']}).json()
  
    fileCount+=1
    if 'data' in res.keys():
        return pd.io.json.json_normalize(res['data'],record_path=None,meta=['deviceName','devicelat','devicelon','last_updated']) #ignoring header) #ignoring header
    else:
        return None
    return res

def realTime_to_schedule():
    json_db = get_realTimeData()

    folder = AQIData_location + tk['today']
    Path(folder).mkdir(parents=True,exist_ok=True)
    outFile = folder +"/realTime_aqi_"+ getTime() +".csv"

    if json_db is None:
        pass
    else:
        print("Writing data to " + outFile)
        json_db.to_csv(outFile, sep = ",")
        
schedule.every(1).minutes.do(realTime_to_schedule)
# warnings.filterwarnings("ignore")

while True:
    schedule.run_pending()
    time.sleep(1)

generating token on  2022-08-16


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Writing data to /content/drive/MyDrive/AQIData/2022-08-16/realTime_aqi_16-Aug-2022-01-07.csv
Writing data to /content/drive/MyDrive/AQIData/2022-08-16/realTime_aqi_16-Aug-2022-01-08.csv


KeyboardInterrupt: ignored